In [1]:
#model套件
import torch
from torch import nn as nn
from torch.nn.parameter import Parameter as Parameter
from torch.utils.data import Dataset
from torch.autograd import Variable as Variable
from torchsummary import summary
import torch.nn.functional as F

#聲音處理套件
import librosa
#資料處理套件
import pandas as pd
import numpy as np
#時間套件
import time

from enum import Enum

In [2]:
#存檔位置
ModelMode = "LeNet5_ESC10_kFold"
TrainDataSoundMode = ["_1s", "_3s", "_5s", "_135s", "_531s"]
filePath_Train = ModelMode + TrainDataSoundMode[4] + "_LRdown_Train.txt"
filePath_LastData = ModelMode + TrainDataSoundMode[4] +  "_LRdown_LastData.txt"
modelPathHead = ModelMode + TrainDataSoundMode[4] + "_LRdown"

In [3]:
#音訊資訊CSV檔位置
DATA_CSV_PATH = "D:/Hu_code/ESC-50-master/ESC-50-master/meta/esc50.csv"
#音訊存放位置
DATA_PATH = "D:/Hu_code/ESC-50-master/ESC-50-master/audio/"

def loadDataESC10(data_path, data_csv_path, soundLength):
    dataX_Fold = []
    dataY_Fold = []
    
    #讀取CSV檔
    df = pd.read_csv(data_csv_path)
    #篩選出ESC10作為訓練資料
    df_10 = df[df['esc10']==True]
    #取出類別，去除重複並排序
    classes = df_10['category'].unique()
    class_dict = {i:x for x,i in enumerate(classes)}
    #重新標label
    df_10['target'] = df_10['category'].map(class_dict)

    #計數
    count = 0
    
    for Fold in range(1, 6, 1):
        #初始化
        dataX = []
        dataY = []
        #篩選資料
        sound_df = df_10[df_10.fold == Fold]
        #轉換train資料/添加標籤
        for data in sound_df.itertuples():
            sig, sr = librosa.load(data_path+data[1])
            for i in range((6-soundLength)):
                sound = sig[i*sr : int((i+soundLength)*sr)]
                sound = audio_to_melspectrogram(audio=sound, sr=sr)
                dataX.append(sound)
                dataY.append(data[3])       
            count +=1
            print("{:.3f}".format(100 * count / len(df_10))+"%", end='\r')
        dataX_Fold.append(dataX)
        dataY_Fold.append(dataY)
                
    #通通轉成 torch 的 tensor 形式
    dataX_Fold = torch.Tensor(dataX_Fold)
    dataY_Fold = torch.Tensor(dataY_Fold).type(torch.LongTensor)
    
    #轉維度 FBCHW
    dataX_Fold = torch.reshape(dataX_Fold, (dataX_Fold.shape[0], dataX_Fold.shape[1], 1, dataX_Fold.shape[2], dataX_Fold.shape[3]))
           
    # 完成時打印最終消息
    print("Progress: 100% - Completed")
    return dataX_Fold, dataY_Fold

class conf:
    sr = 44100  #採樣率
    hop_length = 512  #帧移
    fmin = 20  #最低頻率
    fmax = sr // 2  #最高頻率
    n_mels = 128  #濾波器個數
    n_fft = 2048  #FFT窗口長度

#轉頻譜圖
def audio_to_melspectrogram(audio, sr):
    spectrogram = librosa.feature.melspectrogram(y=audio, 
                                                 sr=sr,
                                                 n_mels=128,
                                                 hop_length=512,
                                                 n_fft=2048,
                                                 fmin=20,
                                                 fmax=sr//2)
    spectrogram = librosa.power_to_db(spectrogram)
    return spectrogram

class CustomDatasetESC50(Dataset):
    def __init__(self, inputDataX, inputDataY, fold, isTrain):
        #防止範圍錯誤
        if(fold>5 or fold <1):
            raise ValueError("Fold範圍錯誤")
            
        #資料初始化
        self.dataX = []
        self.dataY = []
        
        # True:訓練資料  False:測試資料
        if isTrain:
            for Fold in range(1, 6, 1):
                if(Fold != fold):
                    for X in inputDataX[Fold-1]:
                        self.dataX.append(X)
                    for Y in inputDataY[Fold-1]:
                        self.dataY.append(Y)
        else:
            self.dataX = inputDataX[fold-1]
            self.dataY = inputDataY[fold-1]

    def __len__(self):
        return len(self.dataX)
    
    def __getitem__(self, index):
        # 根據索引返回單個樣本及其標籤
        data = self.dataX[index]
        label = self.dataY[index]
        
        return data, label

In [4]:
#LeNet5模型
class LeNet(nn.Module):
    def __init__(self, nums_class=10):
        super(LeNet,self).__init__()
        
        #卷基層
        self.conv_layer = nn.Sequential(
            nn.Conv2d(in_channels=1,out_channels=16,kernel_size=5,stride=1,padding=0),
            nn.MaxPool2d(kernel_size=2, stride=2, padding=0),
            nn.Conv2d(in_channels=16,out_channels=32,kernel_size=5,stride=1,padding=0),
            nn.MaxPool2d(kernel_size=2, stride=2, padding=0)
        )
        
        #全連接層
        self.fullconn_layer = nn.Sequential(
            nn.AdaptiveAvgPool2d(1),
            nn.Flatten(),
            nn.Linear(32,120),
            nn.ReLU(),
            nn.Linear(120,84),
            nn.ReLU(),
            nn.Linear(84,nums_class)           
        )
        
    def forward(self,x):
        output = self.conv_layer(x)
        output = self.fullconn_layer(output)
        return output

In [5]:
#訓練模型(全部訓練)
def train_model_TrainAll(trainFold, setLR):
    #用來記錄全部訓練過程的準確度和loss
    train_loss_array = []
    train_accuracy_array = []
    
    #---------------------------------------------------
    #訓練相關參數
    #---------------------------------------------------
    #訓練次數
    num_epochs = 60
    #批次大小
    BATCH_SIZE = 64
    #學習率
    LR = setLR
    #loss方法
    loss_func = nn.CrossEntropyLoss()
    #優化器
    optimizer = torch.optim.Adam(model.parameters(), lr=LR)
    #--------------------------------------------------------------------
    
    #--------------------------------------------------------------------
    #scheduler
    #--------------------------------------------------------------------
    #調度器
    scheduler  = torch.optim.lr_scheduler.StepLR(optimizer, step_size=15, gamma=0.5)
    #--------------------------------------------------------------------
    
    #--------------------------------------------------------------------
    #資料抓入
    #預處理
    #--------------------------------------------------------------------
    #資料抓入
    trainloader = CustomDatasetESC50(inputDataX = dataX,
                               inputDataY = dataY,
                               fold = trainFold,
                               isTrain = True)
    #分批方法
    train_loader = torch.utils.data.DataLoader(trainloader, batch_size = BATCH_SIZE, shuffle = True)
    #--------------------------------------------------------------------
    
    #訓練次數
    for epoch in range(num_epochs):
        #訓練準確度
        correct_train = 0
        #訓練資料總筆數
        total_train = 0
        #批次訓練
        for i, data in enumerate(train_loader):
            #開啟模型訓練
            model.train()
            #將資料放入gpu
            inputs, labels = data
            inputs, labels = inputs.cuda(), labels.cuda()
            #清除梯度
            optimizer.zero_grad()
            #向前傳播
            outputs = model(inputs)
            #計算loss
            train_loss = loss_func(outputs, labels)
            #反向傳播
            train_loss.backward()
            #梯度下降
            optimizer.step()
            #找出信心度最高的判斷
            predicted = torch.max(outputs.data, 1)[1]
            #總筆數加上這次訓練的筆數
            total_train += len(labels)
            #計算判斷正確的筆數
            correct_train += (predicted == labels).float().sum()
        #計算這次訓練的準確度
        train_accuracy = 100 * correct_train / float(total_train)
        #紀錄這次訓練的準確度
        train_accuracy_array.append(train_accuracy)
        #紀錄這次訓練的loss
        train_loss_array.append(train_loss.data)
        
        #印出本次訓練結果
        print('Train Epoch: {}/{} Train_Loss: {} Train_acc: {:.6f}%'.format(epoch+1, num_epochs, train_loss.data, train_accuracy))
        #調度器走一步
        scheduler.step()
        
    #測試準確度
    test_accuracy1 = test_model(inputDataX = dataX1, inputDataY = dataY1, testFold = trainFold)
    test_accuracy3 = test_model(inputDataX = dataX3, inputDataY = dataY3, testFold = trainFold)
    test_accuracy5 = test_model(inputDataX = dataX5, inputDataY = dataY5, testFold = trainFold)
        
    #將gpu資料取回
    train_loss_array = torch.stack(train_loss_array,dim=-1).cpu()
    train_accuracy_array = torch.stack(train_accuracy_array,dim=-1).cpu()
    
    return train_loss_array, train_accuracy_array, test_accuracy1, test_accuracy3, test_accuracy5

In [6]:
#測試模型的準確度
def test_model(inputDataX, inputDataY, testFold):
    #固定模型不改變權重
    model.eval()
    #計數初始化
    correct = 0
    total = 0
    #資料抓取
    testloader = CustomDatasetESC50(inputDataX = inputDataX,
                               inputDataY = inputDataY,
                               fold = testFold,
                               isTrain = False)
    #封裝                                
    test_loader = torch.utils.data.DataLoader(testloader, batch_size=64, shuffle=False)
    
    #不計算梯度 加速計算
    with torch.no_grad():
        for data in test_loader:
            inputs, labels = data
            inputs, labels = inputs.cuda(), labels.cuda()
            
            #預測結果
            outputs = model(inputs)
            
            #獲取預測標籤
            _, predicted = torch.max(outputs.data, 1)
            
            #計算準確度
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    print('Test accuracy of the model: {:.6f}%'.format(100 * correct / total))
    return (100 * correct / total)

In [7]:
#紀錄開始時間
dataload_start_time = time.time()

#資料讀取
print("讀取1秒資料")
dataX1, dataY1 = loadDataESC10(data_path = DATA_PATH,
                            data_csv_path = DATA_CSV_PATH,
                            soundLength = 1)
print("讀取3秒資料")
dataX3, dataY3 = loadDataESC10(data_path = DATA_PATH,
                            data_csv_path = DATA_CSV_PATH,
                            soundLength = 3)
print("讀取5秒資料")
dataX5, dataY5 = loadDataESC10(data_path = DATA_PATH,
                            data_csv_path = DATA_CSV_PATH,
                            soundLength = 5)

#紀錄結束時間
dataload_end_time = time.time()

#計算耗時
dataload_training_time = dataload_end_time - dataload_start_time

#印出耗時
print("Data Loading Time: {:.2f} seconds".format(dataload_training_time))

讀取1秒資料


C:\Users\test\AppData\Local\Temp\ipykernel_4620\2994481965.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_10['target'] = df_10['category'].map(class_dict)


C:\Users\test\AppData\Local\Temp\ipykernel_4620\2994481965.py:43: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at C:\cb\pytorch_1000000000000\work\torch\csrc\utils\tensor_new.cpp:248.)
  dataX_Fold = torch.Tensor(dataX_Fold)
C:\Users\test\AppData\Local\Temp\ipykernel_4620\2994481965.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_10['target'] = df_10['category'].map(class_dict)


Progress: 100% - Completed
讀取3秒資料
Progress: 100% - Completed
讀取5秒資料


C:\Users\test\AppData\Local\Temp\ipykernel_4620\2994481965.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_10['target'] = df_10['category'].map(class_dict)


Progress: 100% - Completed
Data Loading Time: 27.80 seconds


In [8]:
kFoldTrainTime = []
kFoldTrainAccuracy = []
kFoldTestAccuracy1 = []
kFoldTestAccuracy3 = []
kFoldTestAccuracy5 = []

#設定存檔位置
f = open(filePath_Train, "w")
#k-fold訓練
for fold in range(1,6):
    kTLTrainTime = []
    kTLTrainAccuracy = []
    kTLTestAccuracy1 = []
    kTLTestAccuracy3 = []
    kTLTestAccuracy5 = []

    #當前裝置能使用的GPU數量
    num_devices = torch.cuda.device_count()
    print("可用的 GPU 数量:", num_devices)

    #建立模型
    model = LeNet(nums_class = 10)

    #將模型放置到 GPU 
    model = model.cuda()

    #使用 nn.DataParallel 包装模型
    #進行併行處理
    model = nn.DataParallel(model)
    
    #學習率
    lr=0
    for TL in range(3):
        print("----------------------------------------------------------------------------------------------------------------")
        print("fold: ", fold, "TL: ", TL)
        print("----------------------------------------------------------------------------------------------------------------")
        #訓練資料(要用幾秒訓練)
        if(TL==0):
            #1s
            dataX = dataX5
            dataY = dataY5
            lr = 0.0032
        elif(TL==1):
            #3s
            dataX = dataX3
            dataY = dataY3
            lr = 0.0016
        else:
            #5s
            dataX = dataX1
            dataY = dataY1
            lr = 0.0008

        #紀錄訓練開始時間
        start_time = time.time()

        #訓練模型(只訓練分類器)
        train_loss, train_accuracy, test_accuracy1, test_accuracy3, test_accuracy5 = train_model_TrainAll(trainFold=fold, setLR=lr)

        #紀錄訓練結束時間
        end_time = time.time()

        #計算訓練耗時
        training_time = end_time - start_time

        #印出訓練耗時
        print("Training Time: {:.2f} seconds".format(training_time))

        f.write("---------------------------------------------------------------------------------------------------------------\n")
        f.write("訓練過程\n")
        f.write("---------------------------------------------------------------------------------------------------------------\n")
        for i in range(len(train_loss)):
            # 寫入資料
            f.write("Fold: {} TL: {} Train Epoch: {}/{} Train_Loss: {} Train_Acc: {:.6f}%\n".format(fold, TL, i+1, len(train_loss), train_loss[i], train_accuracy[i]))
        f.write("---------------------------------------------------------------------------------------------------------------\n")
        f.write("Test_Accuracy1: {:.6f}% Test_Accuracy3: {:.6f}% Test_Accuracy5: {:.6f}%\n".format(test_accuracy1, test_accuracy3, test_accuracy5))
        f.write("Training Time: {:.2f} seconds\n".format(training_time))
        f.write("---------------------------------------------------------------------------------------------------------------\n")

        #--------------------------------------------------------------------
        
        kTLTrainTime.append(training_time)
        kTLTrainAccuracy.append(train_accuracy[-1])
        kTLTestAccuracy1.append(test_accuracy1)
        kTLTestAccuracy3.append(test_accuracy3)
        kTLTestAccuracy5.append(test_accuracy5)
        
    kFoldTrainTime.append(kTLTrainTime)
    kFoldTrainAccuracy.append(kTLTrainAccuracy)
    kFoldTestAccuracy1.append(kTLTestAccuracy1)
    kFoldTestAccuracy3.append(kTLTestAccuracy3)
    kFoldTestAccuracy5.append(kTLTestAccuracy5)
    
    modelpath = modelPathHead + "_" + str(fold) + "fold.pth"
    #保存模型
    torch.save(model.state_dict(), modelpath)
    
#關閉
f.close()

可用的 GPU 数量: 1
----------------------------------------------------------------------------------------------------------------
fold:  1 TL:  0
----------------------------------------------------------------------------------------------------------------
Train Epoch: 1/60 Train_Loss: 2.5914907455444336 Train_acc: 12.500000%
Train Epoch: 2/60 Train_Loss: 2.092360496520996 Train_acc: 19.062500%
Train Epoch: 3/60 Train_Loss: 1.9639711380004883 Train_acc: 20.937500%
Train Epoch: 4/60 Train_Loss: 1.7036819458007812 Train_acc: 27.812500%
Train Epoch: 5/60 Train_Loss: 1.683591604232788 Train_acc: 27.187500%
Train Epoch: 6/60 Train_Loss: 1.7806516885757446 Train_acc: 30.000000%
Train Epoch: 7/60 Train_Loss: 1.650212049484253 Train_acc: 36.250000%
Train Epoch: 8/60 Train_Loss: 1.7828017473220825 Train_acc: 32.812500%
Train Epoch: 9/60 Train_Loss: 1.5558528900146484 Train_acc: 36.250000%
Train Epoch: 10/60 Train_Loss: 1.5563127994537354 Train_acc: 33.750000%
Train Epoch: 11/60 Train_Loss: 1.586

Train Epoch: 48/60 Train_Loss: 0.40114766359329224 Train_acc: 86.250008%
Train Epoch: 49/60 Train_Loss: 0.4855335056781769 Train_acc: 85.312508%
Train Epoch: 50/60 Train_Loss: 0.4796249270439148 Train_acc: 86.250008%
Train Epoch: 51/60 Train_Loss: 0.46456581354141235 Train_acc: 86.875008%
Train Epoch: 52/60 Train_Loss: 0.4567238688468933 Train_acc: 86.145836%
Train Epoch: 53/60 Train_Loss: 0.4208126664161682 Train_acc: 86.041672%
Train Epoch: 54/60 Train_Loss: 0.4019266963005066 Train_acc: 85.312508%
Train Epoch: 55/60 Train_Loss: 0.3577045798301697 Train_acc: 86.145836%
Train Epoch: 56/60 Train_Loss: 0.4834615886211395 Train_acc: 85.625008%
Train Epoch: 57/60 Train_Loss: 0.5208439826965332 Train_acc: 86.041672%
Train Epoch: 58/60 Train_Loss: 0.3913460075855255 Train_acc: 87.083336%
Train Epoch: 59/60 Train_Loss: 0.3603852391242981 Train_acc: 86.979172%
Train Epoch: 60/60 Train_Loss: 0.3893195390701294 Train_acc: 85.833336%
Test accuracy of the model: 54.250000%
Test accuracy of the mo

Train Epoch: 33/60 Train_Loss: 0.629543125629425 Train_acc: 69.687500%
Train Epoch: 34/60 Train_Loss: 0.6285027265548706 Train_acc: 70.000000%
Train Epoch: 35/60 Train_Loss: 0.7430626749992371 Train_acc: 72.187500%
Train Epoch: 36/60 Train_Loss: 0.654712975025177 Train_acc: 73.750000%
Train Epoch: 37/60 Train_Loss: 0.7412342429161072 Train_acc: 74.062500%
Train Epoch: 38/60 Train_Loss: 0.5313464403152466 Train_acc: 72.187500%
Train Epoch: 39/60 Train_Loss: 0.599159300327301 Train_acc: 72.187500%
Train Epoch: 40/60 Train_Loss: 0.7619925141334534 Train_acc: 71.250000%
Train Epoch: 41/60 Train_Loss: 0.5757021903991699 Train_acc: 75.000000%
Train Epoch: 42/60 Train_Loss: 0.5963630676269531 Train_acc: 75.312500%
Train Epoch: 43/60 Train_Loss: 0.6658155918121338 Train_acc: 73.437500%
Train Epoch: 44/60 Train_Loss: 0.6947422027587891 Train_acc: 75.312500%
Train Epoch: 45/60 Train_Loss: 0.731744647026062 Train_acc: 75.000000%
Train Epoch: 46/60 Train_Loss: 0.5489672422409058 Train_acc: 77.8125

Train Epoch: 17/60 Train_Loss: 0.35316669940948486 Train_acc: 80.000000%
Train Epoch: 18/60 Train_Loss: 0.5841410160064697 Train_acc: 81.062500%
Train Epoch: 19/60 Train_Loss: 0.42556172609329224 Train_acc: 80.312500%
Train Epoch: 20/60 Train_Loss: 0.4590272903442383 Train_acc: 79.750000%
Train Epoch: 21/60 Train_Loss: 0.6309568285942078 Train_acc: 81.000000%
Train Epoch: 22/60 Train_Loss: 0.48023632168769836 Train_acc: 81.937500%
Train Epoch: 23/60 Train_Loss: 0.42275577783584595 Train_acc: 80.312500%
Train Epoch: 24/60 Train_Loss: 0.4493170380592346 Train_acc: 80.875000%
Train Epoch: 25/60 Train_Loss: 0.5035743117332458 Train_acc: 82.375000%
Train Epoch: 26/60 Train_Loss: 0.4060940444469452 Train_acc: 81.000000%
Train Epoch: 27/60 Train_Loss: 0.32323402166366577 Train_acc: 81.750000%
Train Epoch: 28/60 Train_Loss: 0.4228940010070801 Train_acc: 82.125000%
Train Epoch: 29/60 Train_Loss: 0.40486136078834534 Train_acc: 82.250000%
Train Epoch: 30/60 Train_Loss: 0.49351367354393005 Train_a

Train Epoch: 1/60 Train_Loss: 1.2850414514541626 Train_acc: 53.541668%
Train Epoch: 2/60 Train_Loss: 1.1056602001190186 Train_acc: 61.250004%
Train Epoch: 3/60 Train_Loss: 1.0629123449325562 Train_acc: 65.416672%
Train Epoch: 4/60 Train_Loss: 0.7508006691932678 Train_acc: 63.750004%
Train Epoch: 5/60 Train_Loss: 0.8339684009552002 Train_acc: 67.500000%
Train Epoch: 6/60 Train_Loss: 0.9406446218490601 Train_acc: 66.666672%
Train Epoch: 7/60 Train_Loss: 0.7919139266014099 Train_acc: 67.604172%
Train Epoch: 8/60 Train_Loss: 0.7264261841773987 Train_acc: 71.458336%
Train Epoch: 9/60 Train_Loss: 0.8748892545700073 Train_acc: 70.625000%
Train Epoch: 10/60 Train_Loss: 0.8026723265647888 Train_acc: 71.250000%
Train Epoch: 11/60 Train_Loss: 0.7061084508895874 Train_acc: 69.583336%
Train Epoch: 12/60 Train_Loss: 0.7041723132133484 Train_acc: 71.250000%
Train Epoch: 13/60 Train_Loss: 0.5959571599960327 Train_acc: 73.020836%
Train Epoch: 14/60 Train_Loss: 0.7012824416160583 Train_acc: 73.333336%
T

Train Epoch: 51/60 Train_Loss: 0.39001232385635376 Train_acc: 85.687500%
Train Epoch: 52/60 Train_Loss: 0.4509897828102112 Train_acc: 85.812500%
Train Epoch: 53/60 Train_Loss: 0.4063889682292938 Train_acc: 86.375000%
Train Epoch: 54/60 Train_Loss: 0.514818012714386 Train_acc: 86.250000%
Train Epoch: 55/60 Train_Loss: 0.2945033013820648 Train_acc: 86.687500%
Train Epoch: 56/60 Train_Loss: 0.2459622174501419 Train_acc: 86.062500%
Train Epoch: 57/60 Train_Loss: 0.2011040598154068 Train_acc: 86.062500%
Train Epoch: 58/60 Train_Loss: 0.42501208186149597 Train_acc: 87.250000%
Train Epoch: 59/60 Train_Loss: 0.42024385929107666 Train_acc: 86.625000%
Train Epoch: 60/60 Train_Loss: 0.4066496789455414 Train_acc: 86.625000%
Test accuracy of the model: 66.750000%
Test accuracy of the model: 76.666667%
Test accuracy of the model: 77.500000%
Training Time: 9.51 seconds
可用的 GPU 数量: 1
----------------------------------------------------------------------------------------------------------------
fold: 

Train Epoch: 36/60 Train_Loss: 0.48351219296455383 Train_acc: 82.708336%
Train Epoch: 37/60 Train_Loss: 0.5457556247711182 Train_acc: 82.395836%
Train Epoch: 38/60 Train_Loss: 0.3838330805301666 Train_acc: 83.020836%
Train Epoch: 39/60 Train_Loss: 0.5085343718528748 Train_acc: 82.916672%
Train Epoch: 40/60 Train_Loss: 0.41536372900009155 Train_acc: 82.812508%
Train Epoch: 41/60 Train_Loss: 0.417472779750824 Train_acc: 84.166672%
Train Epoch: 42/60 Train_Loss: 0.5529358983039856 Train_acc: 83.854172%
Train Epoch: 43/60 Train_Loss: 0.49359461665153503 Train_acc: 84.583336%
Train Epoch: 44/60 Train_Loss: 0.43841788172721863 Train_acc: 81.875008%
Train Epoch: 45/60 Train_Loss: 0.408454030752182 Train_acc: 83.958336%
Train Epoch: 46/60 Train_Loss: 0.37655049562454224 Train_acc: 84.479172%
Train Epoch: 47/60 Train_Loss: 0.507891058921814 Train_acc: 84.166672%
Train Epoch: 48/60 Train_Loss: 0.4893771708011627 Train_acc: 84.895836%
Train Epoch: 49/60 Train_Loss: 0.4385000765323639 Train_acc: 8

Train Epoch: 20/60 Train_Loss: 0.9836035966873169 Train_acc: 60.937500%
Train Epoch: 21/60 Train_Loss: 0.9756911396980286 Train_acc: 63.750000%
Train Epoch: 22/60 Train_Loss: 0.8891750574111938 Train_acc: 60.937500%
Train Epoch: 23/60 Train_Loss: 1.1806055307388306 Train_acc: 60.937500%
Train Epoch: 24/60 Train_Loss: 0.9012967348098755 Train_acc: 65.937500%
Train Epoch: 25/60 Train_Loss: 0.716153085231781 Train_acc: 66.562500%
Train Epoch: 26/60 Train_Loss: 0.870418131351471 Train_acc: 68.437500%
Train Epoch: 27/60 Train_Loss: 0.8030239939689636 Train_acc: 70.937500%
Train Epoch: 28/60 Train_Loss: 0.8436923623085022 Train_acc: 67.500000%
Train Epoch: 29/60 Train_Loss: 0.8517197370529175 Train_acc: 72.500000%
Train Epoch: 30/60 Train_Loss: 0.9713656306266785 Train_acc: 72.500000%
Train Epoch: 31/60 Train_Loss: 0.9917325377464294 Train_acc: 69.375000%
Train Epoch: 32/60 Train_Loss: 0.629510223865509 Train_acc: 70.937500%
Train Epoch: 33/60 Train_Loss: 0.7130722999572754 Train_acc: 72.500

Train Epoch: 5/60 Train_Loss: 0.8017643690109253 Train_acc: 75.062500%
Train Epoch: 6/60 Train_Loss: 0.8066437840461731 Train_acc: 73.125000%
Train Epoch: 7/60 Train_Loss: 0.5908414721488953 Train_acc: 74.125000%
Train Epoch: 8/60 Train_Loss: 0.7069587111473083 Train_acc: 75.937500%
Train Epoch: 9/60 Train_Loss: 0.410846471786499 Train_acc: 73.687500%
Train Epoch: 10/60 Train_Loss: 0.6323457360267639 Train_acc: 76.562500%
Train Epoch: 11/60 Train_Loss: 0.5781258940696716 Train_acc: 78.750000%
Train Epoch: 12/60 Train_Loss: 0.625350296497345 Train_acc: 77.062500%
Train Epoch: 13/60 Train_Loss: 0.8072457313537598 Train_acc: 79.437500%
Train Epoch: 14/60 Train_Loss: 0.7087936997413635 Train_acc: 79.562500%
Train Epoch: 15/60 Train_Loss: 0.4929545223712921 Train_acc: 75.562500%
Train Epoch: 16/60 Train_Loss: 0.505382776260376 Train_acc: 80.062500%
Train Epoch: 17/60 Train_Loss: 0.5465556979179382 Train_acc: 81.500000%
Train Epoch: 18/60 Train_Loss: 0.5436878204345703 Train_acc: 80.812500%


In [9]:
class FilterMode(Enum):
    Max = "Max"
    Min = "Min"
    Mean = "Mean"
    
def listFilter(data, mode:FilterMode):
    TL = []
    for fold in range(len(data)):
        TL.append(data[fold][len(data[fold])-1])
    
    switch = {
        FilterMode.Max: max(TL),
        FilterMode.Min: min(TL),
        FilterMode.Mean: np.mean(TL)
    }

    return switch.get(mode, -1)

#設定存檔位置
f = open(filePath_LastData, "w")

#整理出訓練完的數值
f.write("---------------------------------------------------------------------------------------------------------------\n")
f.write("訓練結果\n")
f.write("-----------------------------------------------------------------------------------\n")
for fold in range(5):
    for TL in range(3):
        # 寫入資料
        f.write("Fold: {} TL: {} Train_acc: {:.10f}% Test_accuracy1: {:.6f}% Test_accuracy3: {:.6f}% Test_accuracy5: {:.6f}% Training Time: {:.2f} seconds\n".format(fold+1, TL, kFoldTrainAccuracy[fold][TL], kFoldTestAccuracy1[fold][TL], kFoldTestAccuracy3[fold][TL], kFoldTestAccuracy5[fold][TL], kFoldTrainTime[fold][TL]))
    f.write("-----------------------------------------------------------------------------------\n")
    
# 寫入資料[]
f.write("Max Train_acc: {:.6f}% Test_accuracy1: {:.6f}% Test_accuracy3: {:.6f}% Test_accuracy5: {:.6f}%\n".format(listFilter(kFoldTrainAccuracy, FilterMode.Max), listFilter(kFoldTestAccuracy1, FilterMode.Max), listFilter(kFoldTestAccuracy3, FilterMode.Max), listFilter(kFoldTestAccuracy5, FilterMode.Max)))
f.write("Min Train_acc: {:.6f}% Test_accuracy1: {:.6f}% Test_accuracy3: {:.6f}% Test_accuracy5: {:.6f}%\n".format(listFilter(kFoldTrainAccuracy, FilterMode.Min), listFilter(kFoldTestAccuracy1, FilterMode.Min), listFilter(kFoldTestAccuracy3, FilterMode.Min), listFilter(kFoldTestAccuracy5, FilterMode.Min)))
f.write("Avg Train_acc: {:.6f}% Test_accuracy1: {:.6f}% Test_accuracy3: {:.6f}% Test_accuracy5: {:.6f}%\n".format(listFilter(kFoldTrainAccuracy, FilterMode.Mean), listFilter(kFoldTestAccuracy1, FilterMode.Mean), listFilter(kFoldTestAccuracy3, FilterMode.Mean), listFilter(kFoldTestAccuracy5, FilterMode.Mean)))

f.close()